In [2]:
from src.core import DataLoader

In [3]:
from scipy.spatial import distance

In [4]:
dl = DataLoader('data/jokes.csv', 'data/rating.csv')

In [5]:
from scipy.sparse import csr_matrix

In [6]:
from scipy.sparse import csr_matrix
import numpy as np

from sklearn.metrics.pairwise import linear_kernel

In [7]:
df = dl.train_data

In [8]:
df

,id,user_id,joke_id,Rating
0,31030_110,31030,110,2.750
1,16144_109,16144,109,5.094
2,23098_6,23098,6,-6.438
4,18419_134,18419,134,9.375
5,5782_14,5782,14,-1.781
...,...,...,...,...
1092054,9517_132,9517,132,3.156
1092055,27767_118,27767,118,-1.594
1092056,10580_81,10580,81,2.000
1092057,31007_119,31007,119,8.906


In [9]:
N = len(df['user_id'].unique())
M = len(df['joke_id'].unique())

user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
movie_mapper = dict(zip(np.unique(df["joke_id"]), list(range(M))))

user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["joke_id"])))

user_index = [user_mapper[i] for i in df['user_id']]
movie_index = [movie_mapper[i] for i in df['joke_id']]

X = csr_matrix((df["Rating"], (movie_index, user_index)), shape=(M, N))

In [19]:
X.transpose().shape

(40863, 139)

In [10]:
test = dl.test_data

In [16]:
test

,id,user_id,joke_id,Rating
617312,27448_5,27448,5,-2.656
737939,21149_6,21149,6,3.750
381343,31738_139,31738,139,9.750
489951,13644_58,13644,58,6.000
700537,10131_8,10131,8,-9.875
...,...,...,...,...
226480,25495_55,25495,55,0.906
676103,5345_115,5345,115,-1.531
48647,4993_8,4993,8,0.031
291008,30200_7,30200,7,4.688


In [22]:
pop = dl.train_data.groupby('joke_id').mean()

In [24]:
import math

err = 0
err_pop = 0

for r in test[:20].iterrows():
    joke_id = r[1]['joke_id'] - 1
    user_id = r[1]['user_id'] - 1

    sm = 0
    wg = 0

    C = X[joke_id, :].toarray()[0, :]
    D = C.nonzero()[0]
    
    for ind in D.tolist():
        sim = abs(1 / distance.cosine(X[:, ind].toarray()[:, 0], X[:, user_id].toarray()[:, 0]))
        sm += sim * X[joke_id, ind]
        wg += sim
    
    pred = sm / wg
    err += abs(pred - r[1]['Rating'])
    err_pop += abs(pop.loc[joke_id + 1]['Rating'] - r[1]['Rating'])

In [26]:
err / 20

3.774211052293725

In [27]:
err_pop / 20

3.8226606393036606